In [1]:
import numpy as np

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers

In [2]:
# Making sure GPU is present.
gpus = tf.config.experimental.list_physical_devices('GPU')

In [3]:
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((60000, 28, 28), (10000, 28, 28), (60000,), (10000,))

In [4]:
np.unique(y_train, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8),
 array([5923, 6742, 5958, 6131, 5842, 5421, 5918, 6265, 5851, 5949]))

In [5]:
X_train = (X_train.astype("float32") / 255.0)[..., None]
X_test = (X_test.astype("float32") / 255.0)[..., None]

In [6]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((60000, 28, 28, 1), (10000, 28, 28, 1), (60000,), (10000,))

In [ ]:
model = keras.Sequential([
    layers.Input(shape = (28, 28, 1)),
    layers.Conv2D(32, 3, padding="same", activation="relu"),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding="same", activation="relu"),
    layers.MaxPooling2D(),
    
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dense(10, activation="softmax")
])

model.summary()

2026-01-11 10:21:06.562953: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2026-01-11 10:21:06.562982: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2026-01-11 10:21:06.562992: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2026-01-11 10:21:06.563011: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2026-01-11 10:21:06.563026: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 28, 28, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 14, 14, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       401,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 421,642 (1.61 MB)

 Trainable params: 421,642 (1.61 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
model.compile(
    optimizer = keras.optimizers.Adam(),
    loss = keras.losses.SparseCategoricalCrossentropy(),
    metrics = ["accuracy"]
)

In [9]:
class PrintLogs(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        print(f"Epoch {epoch+1}: " +
              ", ".join([f"{k}={v:.4f}" for k, v in logs.items()]))
        
history = model.fit(X_train, y_train, validation_split=0.1, epochs=5, batch_size=128, verbose=2, callbacks=[PrintLogs()])

Epoch 1/5


2026-01-11 10:21:14.446502: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


Epoch 1: accuracy=0.9342, loss=0.2188, val_accuracy=0.9762, val_loss=0.0781
422/422 - 11s - 25ms/step - accuracy: 0.9342 - loss: 0.2188 - val_accuracy: 0.9762 - val_loss: 0.0781
Epoch 2/5
Epoch 2: accuracy=0.9795, loss=0.0655, val_accuracy=0.9847, val_loss=0.0578
422/422 - 7s - 16ms/step - accuracy: 0.9795 - loss: 0.0655 - val_accuracy: 0.9847 - val_loss: 0.0578
Epoch 3/5
Epoch 3: accuracy=0.9839, loss=0.0525, val_accuracy=0.9882, val_loss=0.0487
422/422 - 7s - 16ms/step - accuracy: 0.9839 - loss: 0.0525 - val_accuracy: 0.9882 - val_loss: 0.0487
Epoch 4/5
Epoch 4: accuracy=0.9851, loss=0.0512, val_accuracy=0.9840, val_loss=0.0592
422/422 - 7s - 16ms/step - accuracy: 0.9851 - loss: 0.0512 - val_accuracy: 0.9840 - val_loss: 0.0592
Epoch 5/5
Epoch 5: accuracy=0.9862, loss=0.0507, val_accuracy=0.9878, val_loss=0.0677
422/422 - 7s - 16ms/step - accuracy: 0.9862 - loss: 0.0507 - val_accuracy: 0.9878 - val_loss: 0.0677


In [10]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

Test Loss: 0.0643, Test Accuracy: 0.9844
